In [2]:
import torch

In [3]:
class language_model():
    def __init__(self,data,params):
        self.data = data
        self.model = None
        self.tokenizer = None
        self.max_length = None
        self.params = params
        self.x = None
        self.y = None
        
    def preprocessing(self):
        
        # integer encode sequences of words
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts([data])
        encoded = self.tokenizer.texts_to_sequences([data])[0]
        
        # retrieve vocabulary size
        vocab_size = len(self.tokenizer.word_index) + 1
        print('Vocabulary Size: %d' % vocab_size)
        
        # encode 2 words -> 1 word
        sequences = list()
        for i in range(2, len(encoded)):
            sequence = encoded[i-2:i+1]
            sequences.append(sequence)
        print('Total Sequences: %d' % len(sequences))
        
        # pad sequences
        self.max_length = max([len(seq) for seq in sequences])
        sequences = pad_sequences(sequences, maxlen=self.max_length, padding='pre')
        print('Max Sequence Length: %d' % self.max_length)
        
        # split into input and output elements
        sequences = array(sequences)
        self.x, self.y = sequences[:,:-1],sequences[:,-1]
        self.y = to_categorical(self.y, num_classes=vocab_size)
        

In [5]:
# source text
data = """ Jack and Jill went up the hill\n
        To fetch a pail of water\n
        Jack fell down and broke his crown\n
        And Jill came tumbling after\n """

params = {"activation":"softmax","epochs":500,"verbose":2,"loss":"categorical_crossentropy",
          "optimizer":"adam","metrics":['accuracy']}

In [8]:
lang_model = language_model(data,params)